In [1]:
pip install ipywidgets


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Valent\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install ydata-profiling

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Valent\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
from ydata_profiling import ProfileReport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('Pricing.csv')
df.head(10)

Area  Room  Parking  Warehouse  Elevator                     Address  \
0   63     1     True       True      True                     Shahran   
1   60     1     True       True      True                     Shahran   
2   79     2     True       True      True                      Pardis   
3   95     2     True       True      True               Shahrake Qods   
4  123     2     True       True      True              Shahrake Gharb   
5   70     2     True       True     False  North Program Organization   
6   87     2     True       True      True                      Pardis   
7   59     1     True       True      True                     Shahran   
8   54     2     True       True     False                    Andisheh   
9   71     1     True       True      True      West Ferdows Boulevard   

          Price  Price(USD)  
0  1.850000e+09    61666.67  
1  1.850000e+09    61666.67  
2  5.500000e+08    18333.33  
3  9.025000e+08    30083.33  
4  7.000000e+09   233333.33  
5  2.050000e+09    68333.33  
6  6.000000e+08    20000.00  
7  2.150000e+09    71666.67  
8  4.930000e+08    16433.33  
9  2.370000e+09    79000.00

In [5]:
df.drop('Price(USD)', axis=1, inplace=True)

In [6]:
ProfileReport(df)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
df = df.drop_duplicates()

In [8]:
df['Area'] = df['Area'].str.replace(',','')

C:\Users\Valent\AppData\Local\Temp\ipykernel_4716\2622504588.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Area'] = df['Area'].str.replace(',','')


In [9]:
df['Area'] = df['Area'].astype(float)

C:\Users\Valent\AppData\Local\Temp\ipykernel_4716\990597524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Area'] = df['Area'].astype(float)


In [10]:
df.dropna(inplace=True)

C:\Users\Valent\AppData\Local\Temp\ipykernel_4716\1379821321.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3248 entries, 0 to 3478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Area       3248 non-null   float64
 1   Room       3248 non-null   int64  
 2   Parking    3248 non-null   bool   
 3   Warehouse  3248 non-null   bool   
 4   Elevator   3248 non-null   bool   
 5   Address    3248 non-null   object 
 6   Price      3248 non-null   float64
dtypes: bool(3), float64(2), int64(1), object(1)
memory usage: 136.4+ KB


In [12]:
def batasAtas(data):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    upper = Q3 + 1.5 * IQR

    return upper

df = df[df['Area'] < batasAtas(df['Area'])]
df = df[df['Price'] < batasAtas(df['Price'])]


### Model CatBoost

In [13]:
df = pd.get_dummies(df, columns=['Address'])
df.replace(True, 1, inplace=True)
df.replace(False, 0, inplace=True)

In [14]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Valent\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [15]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Valent\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor

In [17]:
X = df.drop(columns = 'Price')
y = df['Price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [18]:
cb = CatBoostRegressor(random_state= 1 , iterations= 1000, learning_rate= 0.1)

cb.fit(X_train, y_train)

y_test_pred = cb.predict(X_test)
test_score= r2_score(y_test, y_test_pred)

print(test_score)

0:	learn: 2421235612.6258731	total: 142ms	remaining: 2m 21s
1:	learn: 2320972264.7965393	total: 145ms	remaining: 1m 12s
2:	learn: 2220169830.7111859	total: 147ms	remaining: 48.9s
3:	learn: 2134387704.9274976	total: 150ms	remaining: 37.3s
4:	learn: 2069810970.4424522	total: 153ms	remaining: 30.4s
5:	learn: 2009570098.8562834	total: 155ms	remaining: 25.7s
6:	learn: 1958841619.1161067	total: 158ms	remaining: 22.4s
7:	learn: 1910627596.4272680	total: 161ms	remaining: 19.9s
8:	learn: 1867031504.1005027	total: 164ms	remaining: 18s
9:	learn: 1837698055.2454398	total: 166ms	remaining: 16.5s
10:	learn: 1790898422.7485650	total: 169ms	remaining: 15.2s
11:	learn: 1754053457.3415763	total: 172ms	remaining: 14.1s
12:	learn: 1726541524.2283342	total: 175ms	remaining: 13.3s
13:	learn: 1700449043.7541552	total: 178ms	remaining: 12.5s
14:	learn: 1670429847.4132357	total: 181ms	remaining: 11.9s
15:	learn: 1632759324.0657916	total: 184ms	remaining: 11.3s
16:	learn: 1608909423.9844348	total: 186ms	remaini

In [19]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Valent\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [20]:
import joblib

joblib.dump(cb, 'catboost_moodel.joblib')